## Cамостоятельная реализация решающего дерева  

#### Определение дерева

Решающее дерево - **бинарное** дерево, в котором:
1. Каждой *внутренней вершине* $v$ присвоен предикат предсказания: $B_v : \mathbb{X} \rightarrow \{0, 1\}$
2. Каждой *листовой вершине* $v$ присвоен прогноз $C_v: \mathbb{Y}$, где $\mathbb{Y}$ - область значений таргета

Каждый проход дерева начинается из корня. При прохождении очередной вершины мы двигаемся: *вправо*, если $B_v(x) = 1$; *влево*, если $B_v(x) = 0$. 

При достижении листа на объекте $x$, прогнозом для него будет являться $C_v$

Особенности решающего дерева:
1. Полученная функция кусочно-постоянная $\rightarrow$ **не получится применить градиентные методы**
2. Дерево **не может экстраполировать данные** за пределы уже имеющейся области значений признаков обучающей выборки
3. Дереву свойственно **переобучение**

#### Решающий пень

Дерево можно разбить на составляющие - решающие пни. Они будут представлять собой одну и вершину и два дочерних листа.

Вершину мы будем разделять на листы при помощи предиката $B_{j, t}(x_i)$ . Качество разбиения мы будем оценивать при помощи критерия ветвления $Branch$.

На листьях подзнее мы можем принять решение о необходимости дальнейшего разбиения -> построения еще одного решающего пня.


#### Сложность решающего пня

Пусть у нас есть матрица значений признаков $X \in \mathbb{R}^{D \times N}$ и вектор таргетов $Y \in \mathbb{R}^N$.

В основе вершины пня будет находится разделяющий предикат:
$$B_{j, t}(x_i) = \mathbb{I}\left[x_{ij} \le t\right]$$
Мы будем проходить не по самим значениям признаков, а по средним между значениями.
$$x_i < t_i \le x_{i+1}$$
Поэтому мы пройдем всего по $N-1$ значению каждого признака.

Тогда решение на пне примет вид:
$$(j_{opt}, t_{opt}) = \arg\min_{j,t} L \left( B_{j, t}, X, y \right)$$

Для того чтобы рассчитать $loss$, необходимо еще одного прохождение по $N$, в результате получим, что полный алгоритм решающего пня будет выполняться за $O(DN^2)$, где $D$ - кол-во признаков, $N$ - кол-во объектов.

#### Главная проблема решающих деревьев

Запустив предложенный выше алгоритм рещающего пня рекурсивно, он будет выполняться до тех пор, пока полностью не выучит обучающую выборку -> переобучится.

Если мы поставим задачу найти оптимальное решающее дерево при минимальном количестве разбиений, то решение такой задачи не сможем найти за полиномиальное время, т. к. она относится к np - полным задачам.

Чтобы решить ситуацию в настоящий момент пользуются двумя способами:
1. Жадный алгоритм
2. Оптимизация исходного алгоритма ассимптотически и в константу раз

#### Жадный алгоритм *построения* решающего дерева

У нас уже есть матрица значений признаков $X$, определенная выше. Пусть $X_m \subset X$ - множество всех объектов попавших в текущий лист.

1. Создаем вершину $v$
2. **Если**: выполнен ли критерий остановки $Stop(X_m)$, **то** останавливаемся и ставим ответ $Answ(X_m)$, объявив вершину листом.
3. **Иначе**: Находим предикат $B_{j, t}$ имеющий лучшее разбиение на листы $X_m \rightarrow X_l, X_r$. Максимизируя критерий ветвления $Branch(X_m)$
4. Рекурсивно выполняем алгоритм для листьев $X_l, X_r$

Подробнее разберем каждую функцию представленную в алгоритме:
1. $Stop(X_m)$ - критерий остановки. Необходим для того, чтобы при построении решающего дерева мы могли остановиться и не переобучиться.
2. $Answ(X_m)$ - функция вычисляющая ответ для листа, по попавшим в него объектам $X_m$. Может быть:
   - Для задачи *классификации* ответ может быть: ***меткой самого частого класса*** или ***оценкой*** дискрет. ***распределения вероятностей классов*** для объектов в листе.
   - Для задачи *регрессии*: ***средним, медианой или любой другой статистикой***
   - Для любой задачи *простой моделью*: ***линейной функцией, синусойдой или любой другой функцией***
3. $Stop(X_m)$ - критерий остановки при достижении определенных параметров дерево (про регуляризацию деревьев ниже)
4. $Branch(X_m, j, t)$ - критерий ветвления

##### Подробнее про критерий ветвления

Ответы дерева можем закодировать как: 
- $\bar{c} \in \mathbb{R}^k$ для регрессии и классификации
- $\bar{c} = (c_1, ..., c_k) \in \mathbb{R}: \sum_i \bar{c} = 1$ для дискретного распределения вероятностей классов

Предположим что задана функция потерь $L(y_i, c)$. *В момент поиска оптимального разделения* $X_m = X_l \cup X_r$, мы можем *вычислить* для $X_m$ *константный таргет* $c$ (предикт дерева, если бы вершина была терминальной) и связанный с ним значение ф-ии потерь $L$. А именно - константа $c$ должна минимизировать среднее качество $L$.
$$\frac{1}{|X_m|} \sum_{(x_i, y_i) \in X_m} L(y_i, c)$$

Тогда оптимальное значение:
$$H(X_m) = \min_{c \in Y} \frac{1}{|X_m|} \sum_{(x_i, y_i) \in X_m} L(y_i, c)$$

$H(X_m)$ - называется ***информативностью (impurity)***, чем она ниже, тем предикт дерева ближе к *некоторому константному значению*.

Таким же образом можно вычислить информативность всего решающего пня:
1. $X_l$ - объекты попавшие в левый лист
2. $X_r$ - объекты попавшие в правый лист
3. $c_l$, $c_r$ - константы предсказаний в каждом листе для определенного $B_{j, t}(X_m)$

Фукнция потерь всего пня:
$$\frac{1}{|X_m|} \left( \sum_{(x_i, y_i) \in X_l} L(y_i, c_l) + \sum_{(x_i, y_i) \in X_r} L(y_i, c_r) \right)$$

Связать это с информативностью:
$$ \frac{1}{|X_m|} \left( \frac{|X_l|}{|X_l|} \sum_{(x_i, y_i) \in X_l} L(y_i, c_l) + \frac{|X_r|}{|X_r|} \sum_{(x_i, y_i) \in X_r} L(y_i, c_r) \right) = $$
$$ = \frac{|X_l|}{|X_m|} H(X_l) + \frac{|X_r|}{|X_m|} H(X_r) $$

Чтобы получить качество разбиения мы найдем разницу информативностей вершины и получившихся листьев:
$$Branch(X_m, j, t) = |X_m| * H(X_m) - |X_l| * H(X_l) - |X_r| * H(X_r)$$


Функция потерь выбирается под конкретную задачу, подробнее про выбранные функции будет при их реализации

#### Регуляризация решающих деревьев

Так как дерево обязательно переобучится, если его не ограничить, то необходимо упомянуть о методах регуляризации деревьев:
1. Ограничения по максимальной глубине
2. Ограничения на минимальное количество объектов в листе
3. Ограничение на максимальное количество листьев в дереве
4. Требование, чтобы функционал качества при делении текущей подвыборки на две улучшался не менее чем на $s\%$

Перечисленные действия возможно выполнить на разных этапах действия алгоритма:
- Pre-pruning: В процессе построения дерева при достижении критерия остановки
- Post-pruning: После построения дерева, удалить некоторые вершины так, чтобы качество упало не сильно. Проверяя качество на val

#### Реализация класса обучения

In [2]:
import torch
import torch.nn.functional as F

class SelfDecisionTreeClassifier():
    '''
    Decision Tree Classifier
    
    Criterion: Gini

    Regularization presented with:
    - Max depth of tree
    - Min_leaf_samples of tree

    
    The decisive stump predicate is loking best split on all features all values
    So to optimize all calculations we will use PyTorch and GPU
    '''
    def __init__(self, device: str, criterion: str = 'gini', max_depth: int | None = None, min_leaf_samples: int | None = 1):
        assert criterion in ('gini', 'entropy', 'log_loss')

        self.device = device
        
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_leaf_samples = min_leaf_samples

        self.num_classes = 0
        self.__thresholds = []
        self.__features = []
        self.__is_leaf = []
        
        self.N = 0
        self.D = 0
        

    def __stop(self):
        pass

    def __gini_loss(self):
        pass
        
    def __impurity(self):
        pass

    ## Recursive
    def __split_node(self, cur_idx: int):
        if self.__stop():
            self.__is_leaf[cur_idx] = True
            return
        
        best_score = -float('inf')
        best_feature = None
        best_threshold = None

        for feature_idx in range(D):
            
            sorted_vals, indices = torch.sort(self.X[:feature_idx])
            unique_vals = torch.unique(sorted_vals)

            #continue
        
        
        
        pass
        
    def fit(self, X: np.ndarray, y: np.ndarray):
        assert X.shape[0] == y.shape[0], 'N dimension of X and y must be equal'
        
        self.X = torch.tensor(X, device=self.device)
        self.y = torch.tensor(y, device=self.device)
        
        self.N = X.shape[0]
        self.D = X.shape[1]

        self.num_classes = torch.unique(self.y)
        
        
        
        
    def predict(self):
        pass

NameError: name 'np' is not defined